# Step 3: Human Annotation Pass + QA

Now we annotate. This step covers:
1. Setting up an annotation schema for consistency
2. Using patch views to focus annotation effort
3. Creating and editing labels in the FiftyOne App
4. Running a QA pass to catch errors before training

**Why This Matters**: Bad labels create bad models. A disciplined annotation + QA workflow is the difference between a model that works and one that fails mysteriously in production.

## Load the Dataset and Selected Batch

In [ ]:
import fiftyone as fo

# Load dataset
dataset = fo.load_dataset("kitti_annotation_tutorial")

# Load the batch we selected for annotation
batch_v0 = dataset.load_saved_view("batch_v0_to_annotate")

print(f"Dataset: {dataset.name}")
print(f"Batch v0: {len(batch_v0)} samples to annotate")

## Define the Annotation Schema

Before anyone touches a label, define the rules. A schema prevents:
- Class drift (annotator A uses "car", annotator B uses "Car")
- Attribute inconsistency (some add "occluded", some don't)
- Taxonomy bloat (adding classes mid-project)

For KITTI, we'll use the standard classes:

In [ ]:
# Define our annotation schema
ANNOTATION_SCHEMA = {
    "classes": [
        "Car",
        "Pedestrian", 
        "Cyclist",
        "Truck",
        "Van",
        "Tram",
        "Person_sitting",
        "Misc",
        "DontCare"  # For ambiguous regions
    ],
    "attributes": {
        "occluded": {
            "type": "categorical",
            "values": ["none", "partial", "heavy"]
        },
        "truncated": {
            "type": "boolean"
        }
    }
}

print("Annotation Schema:")
print(f"  Classes: {ANNOTATION_SCHEMA['classes']}")
print(f"  Attributes: {list(ANNOTATION_SCHEMA['attributes'].keys())}")

In [ ]:
# Store schema as dataset info for reference
dataset.info["annotation_schema"] = ANNOTATION_SCHEMA
dataset.save()

print("Schema saved to dataset.info['annotation_schema']")

## Create Patch Views for Efficient Annotation

Patch views let annotators focus on individual objects instead of scrolling through full images. This is especially useful for:
- **Reviewing existing detections** - One patch per detection for quick verification
- **Fixing localization** - Easier to see box boundaries on cropped patches
- **Class verification** - Quick yes/no decisions per object

Let's create a patch view of our selected batch.

In [ ]:
# Create a patches view for the ground_truth detections in our batch
patches_view = batch_v0.to_patches("ground_truth")

print(f"Patches view: {len(patches_view)} object patches")
print(f"From {len(batch_v0)} images")

In [ ]:
# Launch the App with patches view
session = fo.launch_app(patches_view)

## Human Annotation Workflow in the App

### Creating New Labels

To add labels in the FiftyOne App:

1. **Enter Annotate Mode**: Click the pencil icon in the sample modal
2. **Select a label field**: Choose `human_labels` (or create it if it doesn't exist)
3. **Draw bounding boxes**: Click and drag to create detection boxes
4. **Assign class**: Select from the schema-defined classes
5. **Add attributes**: Set occlusion, truncation as needed
6. **Save**: Changes persist automatically

### Editing Existing Labels

To fix existing `ground_truth` labels:
1. Click on a detection to select it
2. Drag corners to adjust the bounding box
3. Click the class label to change it
4. Use the delete key to remove incorrect detections

### Best Practices

- **Be consistent**: Follow the schema strictly
- **When in doubt, DontCare**: Ambiguous objects get the DontCare class
- **Tight boxes**: Boxes should be as tight as possible without cutting off the object
- **Occluded objects**: Still annotate them, but mark the occlusion attribute

## Simulate Annotation (For Tutorial Purposes)

In a real workflow, you'd spend time in the App annotating. For this tutorial, we'll simulate the annotation process by copying ground_truth to human_labels with some modifications.

In [ ]:
# For tutorial: Copy ground_truth to human_labels for the selected batch
# In real usage, this field would be populated by human annotation in the App

for sample in batch_v0:
    if sample.ground_truth:
        # Clone the detections
        human_dets = []
        for det in sample.ground_truth.detections:
            human_det = fo.Detection(
                label=det.label,
                bounding_box=det.bounding_box,
                confidence=1.0  # Human labels have confidence 1.0
            )
            human_dets.append(human_det)
        
        sample["human_labels"] = fo.Detections(detections=human_dets)
    else:
        sample["human_labels"] = fo.Detections(detections=[])
    
    # Update annotation status
    sample["annotation_status"] = "annotated"
    sample.save()

print(f"Created human_labels for {len(batch_v0)} samples")

In [ ]:
# Tag samples as annotated for Batch v0
batch_v0.tag_samples("annotated:v0")

# Remove the 'to_annotate' tag since we're done
batch_v0.untag_samples("to_annotate")

print("Updated tags: removed 'to_annotate', added 'annotated:v0'")

## QA Pass: Verify Label Quality

Before training, run a QA pass to catch systematic errors. We'll check for:

1. **Missing labels**: Samples that should have detections but don't
2. **Class distribution anomalies**: Unexpected class frequencies
3. **Bounding box issues**: Boxes that are too small, too large, or malformed
4. **Golden set verification**: Extra scrutiny on the QA samples

In [ ]:
# QA Check 1: Samples with no detections
from fiftyone import ViewField as F

no_labels = batch_v0.match(F("human_labels.detections").length() == 0)
print(f"QA Check 1 - Samples with no human_labels: {len(no_labels)}")

if len(no_labels) > 0:
    print("  Review these samples - they may need annotation or are legitimately empty")

In [ ]:
# QA Check 2: Class distribution
from collections import Counter

human_labels_list = []
for sample in batch_v0:
    if sample.human_labels:
        human_labels_list.extend([det.label for det in sample.human_labels.detections])

label_counts = Counter(human_labels_list)
print(f"\nQA Check 2 - Class distribution in human_labels:")
for label, count in sorted(label_counts.items(), key=lambda x: -x[1]):
    print(f"  {label}: {count}")

# Flag if any unexpected classes appear
expected_classes = set(ANNOTATION_SCHEMA['classes'])
actual_classes = set(label_counts.keys())
unexpected = actual_classes - expected_classes
if unexpected:
    print(f"\n  WARNING: Unexpected classes found: {unexpected}")
else:
    print("\n  All classes match schema.")

In [ ]:
# QA Check 3: Bounding box sanity checks
import numpy as np

box_areas = []
box_issues = []

for sample in batch_v0:
    if sample.human_labels:
        for det in sample.human_labels.detections:
            x, y, w, h = det.bounding_box
            area = w * h
            box_areas.append(area)
            
            # Flag issues
            if area < 0.001:  # Very small box
                box_issues.append((sample.id, det.id, "tiny_box", area))
            if area > 0.5:  # Very large box (> 50% of image)
                box_issues.append((sample.id, det.id, "huge_box", area))
            if w <= 0 or h <= 0:
                box_issues.append((sample.id, det.id, "invalid_dimensions", (w, h)))

print(f"\nQA Check 3 - Bounding box statistics:")
print(f"  Total boxes: {len(box_areas)}")
print(f"  Mean area: {np.mean(box_areas):.4f}")
print(f"  Min area: {np.min(box_areas):.4f}")
print(f"  Max area: {np.max(box_areas):.4f}")
print(f"  Issues found: {len(box_issues)}")

if box_issues:
    print("\n  Flagged boxes:")
    for sample_id, det_id, issue, value in box_issues[:5]:
        print(f"    Sample {sample_id[:8]}..., Detection {det_id[:8]}...: {issue} ({value})")

In [ ]:
# QA Check 4: Verify golden set samples were annotated
golden_view = dataset.load_saved_view("golden_qa")
golden_in_batch = golden_view.match_tags("batch:v0")

print(f"\nQA Check 4 - Golden set in Batch v0:")
print(f"  Golden samples total: {len(golden_view)}")
print(f"  Golden samples in this batch: {len(golden_in_batch)}")

if len(golden_in_batch) > 0:
    print("  These samples need extra review (they're in the golden QA set)")

## Track Annotation Progress

In [ ]:
# Summary of annotation progress
pool_view = dataset.load_saved_view("active_pool")
annotated = dataset.match(F("annotation_status") == "annotated")
remaining = pool_view.match(F("annotation_status") != "annotated")

print("="*50)
print("ANNOTATION PROGRESS")
print("="*50)
print(f"Pool total:     {len(pool_view)}")
print(f"Annotated:      {len(annotated)} ({100*len(annotated)/len(pool_view):.1f}%)")
print(f"Remaining:      {len(remaining)} ({100*len(remaining)/len(pool_view):.1f}%)")
print(f"\nBatch v0:       {len(batch_v0)} samples")
print(f"Total labels:   {sum(len(s.human_labels.detections) for s in batch_v0 if s.human_labels)}")
print("="*50)

## Summary

In this step, you:

1. **Defined an annotation schema** - Consistent classes and attributes
2. **Created patch views** - Efficient per-object annotation
3. **Annotated Batch v0** - Created `human_labels` field with detections
4. **Ran QA checks**:
   - Missing labels
   - Class distribution
   - Bounding box sanity
   - Golden set verification

**Key Insight**: QA isn't optional. A 5-minute check now saves hours of debugging mysterious model failures later.

**Artifacts Created**:
- `human_labels` field on batch_v0 samples
- `annotated:v0` tag on completed samples
- Schema stored in `dataset.info`

**Next up**: Step 4 - Train Baseline + Evaluate